In [14]:
import sys
sys.path.append(sys.path[0][:sys.path[0].find('DVML-P7') + len('DVML-P7')])
import re
import numpy as np
import pandas as pd
from Code.UtilityFunctions.run_query import run_query
from Code.UtilityFunctions.get_data_path import get_path
from Code.UtilityFunctions.wikidata_functions import wikidata_query, categories_dict_singular, get_city_of_location_with_long_lat
from collections import Counter

In [5]:
location_mappings = pd.read_csv(get_path('location_mappings.csv'))

In [11]:
biz = pd.read_json(get_path("yelp_academic_dataset_business.json"), lines=True)

coordinates = (biz["longitude"].apply(round, args=(2,)).astype(str) + "," + biz["latitude"].apply(round, args=(2,)).astype(str)).unique()
coordinates

array(['-119.71,34.43', '-90.34,38.55', '-110.88,32.22', ...,
       '-90.05,30.02', '-90.23,38.66', '-75.41,40.2'], dtype=object)

In [19]:
coordinates_df = pd.DataFrame(coordinates, columns=['coordinates'])

In [20]:
df = pd.DataFrame(coordinates_df['coordinates'].apply(lambda x: [x] + list(get_city_of_location_with_long_lat(x))).tolist(), columns=['coordinates','mapped_qid', 'qid_label'])


In [21]:
print(f"Number of unique qid's: {len(df.mapped_qid.unique())}\nNumber of unique city names: {len(df.qid_label.unique())}")

Number of unique qid's: 248
Number of unique city names: 245


In [16]:
print(f"Number of unique qid's: {len(location_mappings.city_qid.unique())}\nNumber of unique city names: {len(location_mappings.cityLabel.unique())}")

Number of unique qid's: 248
Number of unique city names: 245


In [1]:
from Code.UtilityFunctions.run_query import run_query
import pandas as pd
query = """
SELECT DISTINCT ?p
WHERE {
    ?s ?p ?o .
    ?s rdfs:Class schema:UserReview .
}
"""

predicates = run_query(query, as_dataframe=True)
predicates

,p.value
0,http://www.w3.org/2000/01/rdf-schema#Class
1,https://schema.org/url
2,https://schema.org/aggregateRating
3,https://schema.org/about
4,https://schema.org/dateCreated
5,https://schema.org/description
6,https://purl.archive.org/purl/yelp/ontology#cool
7,https://purl.archive.org/purl/yelp/ontology#funny
8,https://purl.archive.org/purl/yelp/ontology#useful


In [ ]:
pd.set_option('display.max_rows', 500)

In [74]:
preds = predicates['p.value']
preds = preds.str.replace('https://schema.org/', 'schema:')
preds = preds.str.replace('http://www.w3.org/2000/01/rdf-schema#', 'rdfs:')
preds = preds.str.replace('https://purl.archive.org/purl/yelp/ontology#', 'yelpont:')
print("\n".join(list(preds)))

rdfs:Class
schema:url
schema:aggregateRating
schema:about
schema:dateCreated
schema:description
yelpont:cool
yelpont:funny
yelpont:useful


/tmp/ipykernel_726841/3632575354.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  preds = preds.str.replace('https://schema.org/', 'schema:')
/tmp/ipykernel_726841/3632575354.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  preds = preds.str.replace('http://www.w3.org/2000/01/rdf-schema#', 'rdfs:')
/tmp/ipykernel_726841/3632575354.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  preds = preds.str.replace('https://purl.archive.org/purl/yelp/ontology#', 'yelpont:')
